<a href="https://colab.research.google.com/github/ricardoV94/ThinkBayesPymc3/blob/master/ThinkBayes_Chapter_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture
pip install arviz

In [0]:
import numpy as np
import pymc3 as pm
import theano.tensor as tt
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns

## 11.1 Back to the Euro problem

In [3]:
with pm.Model() as m_11_1:
    m = pm.Bernoulli('m', p=.5)
    p = pm.math.switch(m, .5, .56)
    like = pm.Binomial('like', n=250, p=p, observed=140)
    trace_m_11_1 = pm.sample(10000)

Sequential sampling (2 chains in 1 job)
BinaryGibbsMetropolis: [m]
100%|██████████| 10500/10500 [00:01<00:00, 7955.28it/s]


In [4]:
p = 1 - np.mean(trace_m_11_1['m'])
odds = p/(1-p)
'B_cheat', odds

('B_cheat', 6.183908045977012)

### 11.2 Making a fair comparison

In [5]:
with pm.Model() as m_11_2_1:
    m = pm.Bernoulli('m', p=.5)
    m_sub = pm.Bernoulli('m_sub', p=.5)
    p = pm.math.switch(m, .5, pm.math.switch(m_sub, .4, .6))
    like = pm.Binomial('like', n=250, p=p, observed=140)
    trace_m_11_2_1 = pm.sample(5000)

Sequential sampling (2 chains in 1 job)
BinaryGibbsMetropolis: [m, m_sub]
100%|██████████| 5500/5500 [00:01<00:00, 4576.80it/s]


In [6]:
p = 1 - np.mean(trace_m_11_2_1['m'])
odds = p/(1-p)
'B_two', odds

('B_two', 1.3282887077997672)

In [7]:
with pm.Model() as m_11_2_2:
    m = pm.Bernoulli('m', p=.5)
    p_uniform = pm.Uniform('p_uniform')
    p = pm.math.switch(m, .5, p_uniform)
    like = pm.Binomial('like', n=250, p=p, observed=140)
    trace_m_11_2_2 = pm.sample(5000, step_kwargs={'nuts':{'target_accept':.9}}) # Pseudo-priors should also solve sampling divergences

Sequential sampling (2 chains in 1 job)
CompoundStep
>BinaryGibbsMetropolis: [m]
>NUTS: [p_uniform]
100%|██████████| 5500/5500 [00:08<00:00, 672.37it/s]
The number of effective samples is smaller than 25% for some parameters.


In [8]:
p = 1-np.mean(trace_m_11_2_2['m'])
odds = p/(1-p)
'B_uniform', odds

('B_uniform', 0.48323939483832684)

### 11.3 The triangle prior

In [9]:
with pm.Model() as m_11_3:
    m = pm.Bernoulli('m', p=.5)
    p_triangle = pm.Triangular('p_triangle')
    p = pm.math.switch(m, .5, p_triangle)
    like = pm.Binomial('like', n=250, p=p, observed=140)
    trace_m_11_3 = pm.sample(5000, tune=2000, step_kwargs={'nuts':{'target_accept':.9}}) # Pseudo-priors should also solve sampling divergences

Sequential sampling (2 chains in 1 job)
CompoundStep
>BinaryGibbsMetropolis: [m]
>NUTS: [p_triangle]
100%|██████████| 7000/7000 [00:07<00:00, 962.65it/s]
The number of effective samples is smaller than 25% for some parameters.


In [10]:
p = 1-np.mean(trace_m_11_3['m'])
odds = p/(1-p)
'B_triangle', odds

('B_triangle', 0.8358729575913348)